In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
pd.set_option('display.max_columns', None) #To Display all columns
bankData = pd.read_csv('../input/marketing-banking-file/bank-additional-full.csv')

Attribute Information:<br>Input variables:<br># bank client data:<br>1 - age (numeric)<br>2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')<br>3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)<br>4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')<br>5 - default: has credit in default? (categorical: 'no','yes','unknown')<br>6 - housing: has housing loan? (categorical: 'no','yes','unknown')<br>7 - loan: has personal loan? (categorical: 'no','yes','unknown')<br># related with the last contact of the current campaign:<br>8 - contact: contact communication type (categorical: 'cellular','telephone')<br>9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')<br>10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')<br>11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.<br># other attributes:<br>12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)<br>13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)<br>14 - previous: number of contacts performed before this campaign and for this client (numeric)<br>15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')<br># social and economic context attributes<br>16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)<br>17 - cons.price.idx: consumer price index - monthly indicator (numeric)<br>18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)<br>19 - euribor3m: euribor 3 month rate - daily indicator (numeric)<br>20 - nr.employed: number of employees - quarterly indicator (numeric)<br>
<b>Output variable (desired target):<br></b>
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')

### Initial Data Exploration

In [3]:
bankData.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [4]:
bankData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

- It seems there are no values which are missing in bank marketing data set.

In [5]:
bankData.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,41188.00000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000
mean,40.02406,258.285010,2.567593,962.475454,0.172963,0.081886,93.575664,-40.502600,3.621291,5167.035911
std,10.42125,259.279249,2.770014,186.910907,0.494901,1.570960,0.578840,4.628198,1.734447,72.251528
min,17.00000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.00000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.00000,180.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.00000,319.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.00000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


### Data Cleaning

Columns emp.var.rate,cons.price.idx,cons.conf.idx,euribor.3m and nr.employed does not seem to be meaningful.Converting them to appropiate names

In [6]:
bankData.rename({'emp.var.rate':'Employment_Variation_Rate',
                'cons.price.idx': 'Consumer_Price_Index',
                 'cons.conf.idx':'Consumer_Confidence_Index',
                 'euribor3m':'Euribor_3_Month_Rate',
                 'nr.employed':'No_Employees',
                 'y':'Target_Variable'
                },axis=1,inplace=True)

In [7]:
bankData.columns


Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'Employment_Variation_Rate',
       'Consumer_Price_Index', 'Consumer_Confidence_Index',
       'Euribor_3_Month_Rate', 'No_Employees', 'Target_Variable'],
      dtype='object')

As mentioned earlier <i>Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.</i><br>

In [8]:
#Dropping duration column
bankData.drop('duration',axis=1,inplace=True)

In [9]:
#Checking if duration is dropped
bankData.shape

(41188, 20)

Month and day of the week are not required as it is not required to know what was last month,day of week when the customer was contacted

In [10]:
#Dropping month and day_of_week
bankData.drop(columns = ['month','day_of_week'],axis=1,inplace=True)

In [11]:
#Checking if month and day of week is dropped.
bankData.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'campaign', 'pdays', 'previous', 'poutcome',
       'Employment_Variation_Rate', 'Consumer_Price_Index',
       'Consumer_Confidence_Index', 'Euribor_3_Month_Rate', 'No_Employees',
       'Target_Variable'],
      dtype='object')

### Handling Missing Values

In [12]:
#Checking for missing values
bankData.isnull().sum()

age                          0
job                          0
marital                      0
education                    0
default                      0
housing                      0
loan                         0
contact                      0
campaign                     0
pdays                        0
previous                     0
poutcome                     0
Employment_Variation_Rate    0
Consumer_Price_Index         0
Consumer_Confidence_Index    0
Euribor_3_Month_Rate         0
No_Employees                 0
Target_Variable              0
dtype: int64

- There dosen't seem to be any values missing.
- For categorical columns where it is unknown it looks like a seperate category was created for such cases.It might have happened that the client might not have shared the information.(MNAR)
- In future it might happen that a new client comes and there are some columns where some values are categorized as unknown.Here unknown acts like a valid category.
- So, when we convert these categorical to dummy variables it would automatically take into consideration of unkown and the appropiate value would be assigned.
